In [ ]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


In [ ]:
#download the json file from internet
#read json file to get geographical data
india_json= json.load(open("states_india.geojson",'r'))


In [ ]:
india_json['features'][10]

In [ ]:
# read the datafram list  store in dataframe df
df=pd.read_csv("india_data.csv")

# Data cleaning

In [ ]:
# deleted unwanted columns
df.drop(["National Share (%)","Percent rural","Percent urban"],axis=1,inplace=True)


In [ ]:
#clean the density column

df["Density"]= df["Density[a]"].apply(lambda x:int(float(x.split("/")[0].replace(",",""))))

In [ ]:
# Mapping geojson file with dataframe
state_id_map={}
for feature in india_json['features']:
    #create a id column to do mapping of two files
    feature['id']= feature['properties']['state_code']
    #link id column with state name column
    state_id_map[feature['properties']['st_nm']]= feature['id']


In [ ]:
df["State or union territory"].replace({"Jammu and Kashmir": "Jammu & Kashmir", "Manipur[d]": "Manipur","Arunachal Pradesh": "Arunanchal Pradesh","Dadra and Nagar Haveli and Daman and Diu": "Daman & Diu", "Andaman and Nicobar Islands": "Andaman & Nicobar Island"}, inplace=True)


In [ ]:
# remove a ladakh and india from column
df.drop(df.index[df["State or union territory"] == 'Ladakh'], inplace = True)
df.drop(df.index[df["State or union territory"] == 'India'], inplace = True)


In [ ]:
# add id colum in dataframe
df['id']= df["State or union territory"].apply(lambda x:state_id_map[x])

In [ ]:
 # use of log10() to get a more clrity of graph over Density column
df["DensityScale"]=np.log10(df["Density"])

In [ ]:
#change the data type of population column and did some cleaning
df["Population"] = df["Population"].apply(lambda x:int(float(x.split("(")[0].replace(",","").replace("[b]",""))))


# Visualization of dataset

In [ ]:
# choroplath
fig= px.choropleth(df,locations="id",
                   geojson=india_json,
                   color="DensityScale",
                   scope="asia",
                   hover_name="State or union territory",
                   hover_data=["Density"])
fig.update_geos(fitbounds="locations", visible= False)
fig.show()

In [ ]:
# Choroplath mapbox

fig= px.choropleth_mapbox(df,locations="id",
                   geojson=india_json,
                   color="DensityScale",
                    hover_name="State or union territory",
                   hover_data=["Density"],
                   mapbox_style="carto-positron",
                    center={"lat":25 , "lon":78},
                   zoom=3,opacity= 0.6)
fig.show()

In [ ]:
df["Population"].dtypes

In [ ]:
df.head()

In [ ]:
# choroplath
fig= px.choropleth(df,locations="id",
                   geojson=india_json,
                   color="Population",
                   scope="asia",
                   hover_name="State or union territory",
                   hover_data=["Population"],
                  color_continuous_scale=px.colors.sequential.Viridis)
fig.update_geos(fitbounds="locations", visible= False)
fig.show()

In [ ]:
fig = px.bar(df, y="State or union territory", x="Population", color="Population", orientation="h",
             color_continuous_scale='Bluered_r')

fig.show()

In [ ]:
# together view of urban and rural population using pie chart

fig= px.pie(df,names="State or union territory",values="Urban population" , hover_data=["Rural population"])
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [ ]:
df.head()

In [ ]:
# seprate view of urban and rural population 
#Create subplots: use 'domain' type for Pie subplot

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df["State or union territory"], values=df["Urban population"], name="Urban population"),
              1, 1)
fig.add_trace(go.Pie(labels=df["State or union territory"], values=df["Rural population"], name="Rural population"),
                            1, 2)
#to wrap the text inside the pie
fig.update_traces(textposition='inside', textinfo='percent+label')

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Urban and Rural population in India",
    
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Urban', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Rural', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()